# Ballot Polling Assertion RLA

## Overview of the assertion audit tool

The tool requires as input:

+ audit-specific and contest-specific parameters, such as
    - whether to sample with or without replacement
    - the name of the risk function to use, and any parameters it requires
    - a risk limit for each contest to be audited
    - the social choice function for each contest, including the number of winners
    - candidate identifiers
+ a ballot manifest**
+ a random seed
+ reported results for each contest
+ json files of assertions for IRV contests (one file per IRV contest)
+ human reading of voter intent from the paper cards selected for audit

** The ballot manifest could be for cards purported to contain the contests under audit (manifest_type == "STYLE"), or could include cards that might not contain the contest (manifest_type == "ALL"). These are treated differently. If the sample is to be drawn only from cards that--according to the manifest--contain the contest, and a sampled card turns out not to contain the contest, that is considered a discrepancy, dealt with using the "phantoms to zombies" approach. It is assumed that every card in the manifest corresponds to a ballot casted, but there might be ballots casted with no corresponding manifest entry. In that case, phantom records are created to ensure that the audit is still truly risk-limiting. 

Given an independent (i.e., not relying on the voting system) upper bound on the number of cards that contain the contest, if the number of manifest entries that contain the contest does not exceed that bound, we can sample from paper purported to contain the contest and use the "zombies" approach (Banuelos & Stark) to deal with missing manifest entries. This can greatly increase the efficiency of the audit if the contest is on only a small percentage of the cast cards.

Any sampled phantom card (i.e., a card for which there is no manifest entry) is treated as if its MVR was least favorable (a "zombie" producing the greatest doubt in every assertion, separately). Any sampled manifest entry for which there is no corresponding MVR that contains the contest is treated in the least favorable way for each assertion (i.e., as a zombie rather than a non-vote). The least favorable outcome for a ballot in a ballot polling audit is a valid vote for every loser. 

This tool helps select cards for audit, and reports when the audit has found sufficient strong evidence to stop. 

The tool exports a log of all the audit inputs, including the auditors' manually determined voter intent from the audited cards. 

The current version uses a single sample to audit all contests. It is possible to refine things to target smaller contests. 

In [1]:
from __future__ import division, print_function

import math
import json
import warnings
import numpy as np
import pandas as pd
import csv
import copy

from collections import OrderedDict
from IPython.display import display, HTML

from cryptorandom.cryptorandom import SHA256
from cryptorandom.sample import sample_by_index

from assertion_audit_utils import \
    Assertion, Assorter, CVR, TestNonnegMean, check_audit_parameters,\
    find_p_values, find_sample_size, new_sample_size, summarize_status,\
    write_audit_parameters
from dominion_tools import \
    prep_dominion_manifest, sample_from_manifest, write_cards_sampled

# Audit parameters.

* `seed`: the numeric seed for the pseudo-random number generator used to draw sample 
* `replacement`: whether to sample with replacement. If the sample is drawn with replacement, gamma must also be specified.
* `risk_function`: the function to be used to measure risk. Options are `kaplan_markov`,`kaplan_wald`,`kaplan_kolmogorov`,`wald_sprt`,`kaplan_martingale`. Not all risk functions work with every social choice function. `wald_sprt` applies only to plurality contests.
* `g`: a parameter to hedge against the possibility of observing a maximum overstatement. Require $g \in [0, 1)$ for `kaplan_kolmogorov`, `kaplan_markov`, and `kaplan_wald`.
* `N_cards`: an upper bound on the number of pieces of paper cast in the contest. This should be derived independently of the voting system. A ballot consists of one or more cards.

----

* `manifest_file`: filename for ballot manifest (input)
* `manifest_type`: "STYLE" if the manifest is supposed to list only cards that contain the contests under audit; "ALL" if the manifest contains all cards cast in the election
* `assertion_file`: filename of assertions for IRV contests, in RAIRE format (input)
* `sample_file`: filename for sampled card identifiers (output)
* `mvr_file`: filename for manually ascertained votes from sampled cards (input)
* `log_file`: filename for audit log (output)

----

* `contests`: a dict of contest-specific data 
    + the keys are unique contest identifiers for contests under audit
    + the values are dicts with keys:
        - `risk_limit`: the risk limit for the audit of this contest
        - `choice_function`: `plurality`, `supermajority`, or `IRV`
        - `n_winners`: number of winners for majority contests. (Multi-winner IRV not supported; multi-winner super-majority is nonsense)
        - `share_to_win`: for super-majority contests, the fraction of valid votes required to win, e.g., 2/3.
        - `candidates`: list of names or identifiers of candidates
        - `reported_winners` : list of identifier(s) of candidate(s) reported to have won. Length should equal `n_winners`.
        - `assertion_file`: filename for a set of json descriptors of Assertions (see technical documentation) that collectively imply the reported outcome of the contest is correct. Required for IRV; ignored for other social choice functions

In [2]:
seed = 20546205145833673229  # use, e.g., 20 rolls of a 10-sided die. Seed doesn't have to be numeric
replacement = False

#risk_function = "kaplan_martingale"
#risk_fn = lambda x: TestNonnegMean.kaplan_martingale(x, N=N_cards)[0]

risk_function = "kaplan_kolmogorov"
risk_fn = lambda x: TestNonnegMean.kaplan_kolmogorov(x, N=N_cards, t=1/2, g=g)

g = 0.5
N_cards = 300000 # Upper bound on number of ballots cast 


In [3]:
manifest_file = './Data/N19 ballot manifest with WH location for RLA Upload VBM 11-14.xlsx'
manifest_type = 'STYLE' # every card should contain the contest #'ALL' not supported yet
sample_file = './Data/sample_polling.csv'
mvr_file = './Data/mvr_prepilot_test.json'
log_file = './Data/log_polling.json'

In [4]:
# contests to audit. Edit with details of your contest (eg., Contest 339 is the DA race)
contests = {'339':{'risk_limit': 0.05,
                     'choice_function':'IRV',
                     'n_winners':1,
                     'candidates':['15','16','17','18'],
                     'reported_winners' : ['15'],
                     'assertion_file' : './Data/SF2019Nov8Assertions.json'
                    }
           }

Example of other social choice functions:

> contests =  {'city_council':{'risk_limit':0.05,
                     'choice_function':'plurality',
                     'n_winners':3,
                     'candidates':['Doug','Emily','Frank','Gail','Harry'],
                     'reported_winners' : ['Doug', 'Emily', 'Frank']
                    },
            'measure_1':{'risk_limit':0.05,
                     'choice_function':'supermajority',
                     'share_to_win':2/3,
                     'n_winners':1,
                     'candidates':['yes','no'],
                     'reported_winners' : ['yes']
                    }                  
           }

In [5]:
# read the assertions for the IRV contest
for c in contests:
    if contests[c]['choice_function'] == 'IRV':
        with open(contests[c]['assertion_file'], 'r') as f:
            contests[c]['assertion_json'] = json.load(f)['audits'][0]['assertions']

In [6]:
# construct the dict of dicts of assertions for each contest
all_assertions = Assertion.make_all_assertions(contests)

## Process the ballot manifest

In [7]:
# Read manifest
manifest = pd.read_excel(manifest_file)

# Add phantoms if necessary
manifest, manifest_cards, phantom_cards = prep_dominion_manifest(manifest, N_cards)

manifest


,Tray #,Tabulator Number,Batch Number,Total Ballots,VBMCart.Cart number,cum_cards
0,1,99808,78,116,3,116
1,1,99808,77,115,3,231
2,1,99808,79,120,3,351
3,1,99808,81,76,3,427
4,1,99808,80,116,3,543
...,...,...,...,...,...,...
5477,3506,99815,84,222,19,292779
5478,3506,99815,83,346,19,293125
5479,3506,99815,82,332,19,293457
5480,3507,99802,822,98,14,293555


In [8]:
check_audit_parameters(risk_function, g, contests)

In [9]:
write_audit_parameters(log_file, seed, replacement, risk_function, g, \
                        contests, N_cards, manifest_cards, phantom_cards)


## Set up for sampling

## Initial sample size

In [10]:
# TODO: Use initial_sample_size function to calculate
# w/ alpha=0.05, winning proportion ~ 0.58 (ASN formula)
sample_size = 200

## Draw the first sample

In [11]:
prng = SHA256(seed)
sample = sample_by_index(N_cards, sample_size, prng=prng)
n_phantom_sample = np.sum([i>manifest_cards for i in sample]) 
print("The sample includes {} phantom cards.".format(n_phantom_sample))

The sample includes 1 phantom cards.


In [12]:
manifest_sample_lookup = sample_from_manifest(manifest, sample)

In [13]:
write_cards_sampled(sample_file, manifest_sample_lookup, print_phantoms=True)

## Read the audited sample data

In [14]:
with open(mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

for i in range(10):
    print(mvr_sample[i])

id: 99807-3-2 votes: {'339': {'16': 3, '17': 2, '18': 1}} phantom: False
id: 99809-27-41 votes: {'339': {'15': 3, '16': 1, '17': 4, '18': 2}} phantom: False
id: 99807-4-20 votes: {'339': {'15': 1, '17': 2}} phantom: False
id: 99805-68-45 votes: {'339': {'15': 4, '16': 1, '17': 2, '18': 3}} phantom: False
id: 99805-30-44 votes: {'339': {'15': 3, '16': 2, '17': 1, '18': 4}} phantom: False
id: 99805-30-89 votes: {'339': {'15': 2, '17': 1}} phantom: False
id: 99808-28-57 votes: {'339': {'17': 1}} phantom: False
id: 99811-26-37 votes: {'339': {'18': 1}} phantom: False
id: 99804-19-38 votes: {'339': {'15': 2, '18': 1}} phantom: False
id: 99802-15-23 votes: {'339': {'15': 3, '16': 4, '17': 1, '18': 2}} phantom: False


## Find measured risks for all assertions

In [15]:
p_max = find_p_values(contests, all_assertions, risk_fn, manifest_type, mvr_sample)
print("maximum assertion p-value {}".format(p_max))
done = summarize_status(contests, all_assertions)

maximum assertion p-value 1.0
p-values for assertions in contest 339
18 v 17 elim 15 16 45 0.44444592593086424
17 v 16 elim 15 18 45 0.16442065403052356
15 v 18 elim 16 17 45 1
18 v 16 elim 15 17 45 0.18177495204054053
17 v 16 elim 15 45 0.12331302403563205
15 v 17 elim 16 45 0.666671111140741
15 v 17 elim 16 18 45 0.8888933333580249
18 v 16 elim 15 45 0.0006393433634743348
15 v 16 elim 17 45 0.03551102651343496
15 v 16 elim 17 18 45 0.6158738145782814
15 v 16 elim 18 45 0.0006720045032241711
15 v 16 elim 45 0.00043121861510718234
15 v 45 2.694246027008499e-11

contest 339 audit INCOMPLETE at risk limit 0.05. Attained risk 1.0
assertions remaining to be proved:
18 v 17 elim 15 16 45: current risk 0.44444592593086424
17 v 16 elim 15 18 45: current risk 0.16442065403052356
15 v 18 elim 16 17 45: current risk 1
18 v 16 elim 15 17 45: current risk 0.18177495204054053
17 v 16 elim 15 45: current risk 0.12331302403563205
15 v 17 elim 16 45: current risk 0.666671111140741
15 v 17 elim 16 18 4

In [16]:
# Log the status of the audit
write_audit_parameters(log_file, seed, replacement, risk_function, g, \
                        contests, N_cards,  manifest_cards, phantom_cards)

# How many more cards should be audited?

Estimate how many more cards will need to be audited to confirm any remaining contests. The enlarged sample size is based on:

* cards already sampled
* the assumption that we will continue to see errors at the same rate observed in the sample

In [ ]:
new_size, sams = new_sample_size(contests, all_assertions, risk_fn, manifest_type, \
                                    mvr_sample,cvr_sample=None, quantile=0.8, reps=10)
print(new_size, sams)

In [ ]:
# augment the sample
# reset the seed
prng = SHA256(seed)
old_sample = sample
sample = sample_by_index(N_cards, new_size, prng=prng)
incremental_sample = np.sort(list(set(sample) - set(old_sample)))
n_phantom_sample = np.sum([i>manifest_cards for i in incremental_sample])
print("The incremental sample includes {} phantom cards.".format(n_phantom_sample))

In [ ]:
manifest_sample_lookup_new = sample_from_manifest(manifest, incremental_sample)
write_cards_sampled(sample_file, manifest_sample_lookup, print_phantoms=True)

In [ ]:
# mvr_json should contain the complete set of mvrs, including those in previous rounds

with open(mvr_file) as f:
    mvr_json = json.load(f)

mvr_sample = CVR.from_dict(mvr_json['ballots'])

In [ ]:
# compile entire sample
manifest_sample_lookup = sample_from_manifest(manifest, sample)

## Find measured risks for all assertions

In [ ]:
p_max = find_p_values(contests, all_assertions, risk_fn, mvr_sample)
print("maximum assertion p-value {}".format(p_max))
done = summarize_status(contests, all_assertions)

In [ ]:
# Log the status of the audit
write_audit_parameters(log_file, seed, replacement, risk_function, g, \
                        contests, N_cards,  manifest_cards, phantom_cards)